# IMPORTING LIBRARIES

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer 
stemmer = LancasterStemmer()
import time
import difflib
import numpy
import tflearn
import tensorflow
import json
import pickle
import random
import os
import re

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


# INPUT PRE PROCESSING

In [2]:
with open("intent_1.json") as file:
    data = json.load(file)
    
try:
    with open("data.pickle" , "rb") as f:
        words, labels, training, output = pickle.load(f)
        
except:
    words =[]
    labels = []
    docs_patt = []
    docs_tag = []
else:
    words =[]
    labels = []
    docs_patt = []
    docs_tag = []
    
    
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            for item in wrds:
                words.extend(wrds)
                docs_patt.append(wrds)
                docs_tag.append(intent["tag"])
                if intent["tag"] not in labels: 
                    labels.append(intent["tag"])
    words = [stemmer.stem(w.lower()) for w in words] 
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))] 
        # BAG OF WORDS - FEATURE ENGINEERING
    for x, doc in enumerate(docs_patt):
        bag = [] 
        wrds =[stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:] 
        output_row[labels.index(docs_tag[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [3]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])]) 
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output [0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
try:
    model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    history = model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\Prashant Wadile\model.tflearn


In [5]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return numpy.array(bag)

In [6]:
def words_to_list(s):
    a = []
    ns = ""
    s = s + " "
    for i in range(len(s)):
        if s[i] == " ":
            a.append(ns)
            ns = ""
        else:
            ns = ns + s[i]
    a = list(set(a))
    return a

In [7]:
#pass the file in this function to create a dictionary of unique vocabulary
def json_to_dictionary(data):
    dictionary = []
    fil_dict = []
    vocabulary = []
    for i in data["intents"]:
        for pattern in i["patterns"]:
            vocabulary.append(pattern.lower())
    for i in vocabulary:
        dictionary.append(words_to_list(i))
    for i in range(len(dictionary)):
        for word in dictionary[i]:
            fil_dict.append(word)
    return list(set(fil_dict))
    
#for spelling check
chatbot_vocabulary = json_to_dictionary(data)

In [8]:
def word_checker(s):
    correct_string = ""
    for word in s.casefold().split():
        if word not in chatbot_vocabulary:
            suggestion = difflib.get_close_matches(word, chatbot_vocabulary)
            for x in suggestion:
                pass
            if len(suggestion) == 0:
                pass
            else:
                correct_string = correct_string + " " + str(suggestion[0])
        else:
            correct_string = correct_string + " " + str(word)
            
    return correct_string

# SPEECH RECOGNITION

In [9]:
import speech_recognition as sr
r=sr.Recognizer()
import pyttsx3
engine = pyttsx3.init()

In [10]:
def bot_speaking(message):
    engine.say(message)
    engine.runAndWait()
    if engine._inLoop:
        engine.endLoop()

In [11]:
def get_input():
    with sr.Microphone() as source:
        print("Say something!")
        bot_speaking("Hey mate, say something!")
        audio=r.listen(source,timeout=0)
        print("Prefect, Thanks!")
        bot_speaking("Perfect, Thanks!")
    try:
        msg=r.recognize_google(audio)
        print("TEXT: "+msg);
        sr.recognize(audio,Language='hi-IN')
        bot_speaking("you said "+msg)
        return msg
    except:
        print("Dude it's not working ")
        bot_speaking("Sorry mate! It's not working")
        pass;

# BACKEND WORKING

In [14]:
def chat():
    print("UMIT: Hi! I am your personal bot. I am here to answer queries")
    while True:
        #inp=str(get_input())
        inp=input("YOU : ")
        if inp.lower() == "quit" or inp==None:
            break
        inp_x = word_checker(inp)
        results = model.predict([bag_of_words(inp_x, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(results[results_index])
        if results[results_index] >= 0.5:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms= random.choice(responses)
                    pdf = ms.split()
                    for i in pdf:
                        if i.startswith("https:") or i.startswith("http:"):
                            val = True
                        
                        else:
                            val = False
                    if val == True:
                        bot_speaking("Check out this pdf")
                    else:
                        bot_speaking(ms)
                    print("UMIT : " +ms)
                
        else:
            print("UMIT: Sorry, I don't know how to answer that yet")
            bot_speaking("Sorry, I don't know how to answer that yet")
chat()

UMIT: Hi! I am your personal bot. I am here to answer queries
YOU : hi
UMIT : Hi there, what can I do for you?
YOU : fees
UMIT : You can check this out at https://www.umit.ac.in/_files/ugd/db66c0_c7f0b40de0874873b032384eb3aedc4f.pdf
YOU : placements
UMIT : In the year 2021-22, 177 students were placed with 22 companies providing highest package of Rs 19.63 Lakh CTC.
YOU : quit


# GUI INTERFACE / FRONTEND

In [15]:
from tkinter import *

BG_GRAY = "#e7a2f5"
BG_COLOR = "#f5f5f5"
TEXT_COLOR = "#e1195f"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"
bot_name = "UMIT-BOT"


def get_response(msg):
    #print("Hi! I am your personal bot. I am here to answer queries on Usha Mittal Institute of Technology")
    while True:
        inp = msg
        if inp.lower() == "quit" or inp == None:
            break
        inp_x = word_checker(inp)
        results = model.predict([bag_of_words(inp_x, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        # print(results[results_index])
        if results[results_index] >= 0.5:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms = random.choice(responses)
                    return ms
        else:
            return " Sorry, I don't know how to answer that yet "


class ChatApplication:
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()

    def run(self):
        self.window.mainloop()

    def _setup_main_window(self):
        self.window.title("CHATBOT FOR UMIT")
        self.window.resizable(width=True, height=True)
        self.window.configure(width=470, height=550, bg=BG_COLOR)

        # head label
        head_label = Label(
            self.window,
            bg=BG_COLOR,
            fg=TEXT_COLOR,
            text="WELCOME TO UMIT CHAT-BOT",
            font=FONT_BOLD,
            pady=10,
        )
        head_label.place(relwidth=1)

        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)

        # text widget
        self.text_widget = Text(
            self.window,
            width=20,
            height=2,
            bg=BG_COLOR,
            fg=TEXT_COLOR,
            font=FONT,
            padx=5,
            pady=5,
        )
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow")

        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)

        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#fafafa", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)

        # send button
        send_button = Button(
            bottom_label,
            text="Send",
            font=FONT_BOLD,
            width=20,
            bg=BG_GRAY,
            command=lambda: self._on_enter_pressed(None),
        )
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)

    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "YOU")

    def _insert_message(self, msg, sender):
        if not msg:
            return

        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)

        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        bot_speaking(get_response(msg))
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)

        self.text_widget.see(END)


if __name__ == "__main__":
    app = ChatApplication()
    app.run()